In [ ]:
# 법정동 코드 전체에서 5자리(시도+시군구)만 추출하는 코드

In [ ]:
import sys

# -------------------------------------------------------------------------
# [설정] 여기에 다운로드한 '법정동코드 전체자료.txt' 파일의 전체 경로를 입력하세요.
# 예: "C:/Users/user/Downloads/법정동코드 전체자료.txt" (윈도우)
# 예: "/home/user/Downloads/법정동코드 전체자료.txt" (리눅스/Mac)
FILE_PATH = "법정동코드 전체자료.txt"
# -------------------------------------------------------------------------

def parse_region_data(file_path):
    """
    법정동 코드 텍스트 파일에서 시/도 및 시/군/구 데이터를 추출하여
    SQL INSERT 문을 생성하는 함수
    """
    
    # 중복된 5자리 코드를 방지하기 위한 집합(set)
    processed_codes = set()
    
    # 생성된 SQL INSERT 문을 담을 리스트
    sql_statements = []

    print("-- Legal district code data for 'region_codes' table")
    print("START TRANSACTION;")
    print("INSERT INTO region_codes (code, sido, sigungu) VALUES")

    try:
        # 윈도우 환경의 한글 인코딩 'cp949'로 파일을 엽니다.
        with open(file_path, 'r', encoding='cp949') as f:
            # 첫 줄(헤더)은 건너뜁니다.
            next(f)
            
            for line in f:
                try:
                    # 탭으로 각 컬럼을 분리합니다.
                    code_full, full_name, status = line.strip().split('\t')

                    # 1. '폐지'되지 않고 '존재'하는 데이터만 사용합니다.
                    if status != '존재':
                        continue

                    # 2. 앞 5자리 코드만 추출합니다.
                    code_prefix = code_full[:5]
                    
                    # 3. 이미 처리된 5자리 코드는 건너뛰어 중복을 방지합니다.
                    if code_prefix in processed_codes:
                        continue
                    
                    # 4. 법정동명을 시/도와 시/군/구로 분리합니다.
                    name_parts = full_name.split()
                    sido = name_parts[0]
                    sigungu = name_parts[1] if len(name_parts) > 1 else None

                    # 5. SQL INSERT 문 형식에 맞게 데이터를 가공합니다.
                    # sigungu가 없는 경우(시/도 전체)는 NULL로 처리합니다.
                    sigungu_sql = f"'{sigungu}'" if sigungu else "NULL"
                    
                    sql_values = f"('{code_prefix}', '{sido}', {sigungu_sql})"
                    sql_statements.append(sql_values)
                    
                    # 처리된 코드를 기록합니다.
                    processed_codes.add(code_prefix)

                except ValueError:
                    # 간혹 라인 형식이 맞지 않는 경우를 대비한 예외 처리
                    # print(f"Skipping malformed line: {line.strip()}")
                    continue

    except FileNotFoundError:
        print(f"오류: 파일을 찾을 수 없습니다. '{file_path}' 경로를 확인해주세요.")
        return
    except Exception as e:
        print(f"오류 발생: {e}")
        return

    # 마지막 라인을 제외하고 모든 라인 끝에 쉼표(,)를 붙여 출력합니다.
    print(",\n".join(sql_statements))
    print(";")
    print("COMMIT;")


In [ ]:

# 함수를 실행하여 SQL 문을 생성합니다.
if __name__ == "__main__":
    parse_region_data('./법정동코드전체자료.txt')